# Assignment 1: Data Tables

Create an application that displays a DataTable

* Before returning the table to the front end, make it filterable by the price slider and country dropdown menu we developed before.
* Then add options for the user further sort, filter, and export the table.
* Feel free to style it as you like!


In [7]:
from dash import dcc, html, dash_table
from jupyter_dash import JupyterDash
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc

import plotly.express as px
import pandas as pd

ski_resorts = pd.read_csv("../data/European_Ski_Resorts.csv").drop("Unnamed: 0", axis=1)

dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SLATE, dbc_css])

app.layout = html.Div([
    html.H2(id="title", style={"text-align": "center"}),
    html.P("Select Options Below:"),
    dcc.Dropdown(
        id="country-dropdown", 
        options=ski_resorts["Country"].unique(),
        value="Andorra",
        className="dbc"
    ),
    dcc.Slider(
        id="Elevation Slider",
        min=0,
        max=4000,
        step=500,
        value=500,
        marks={
            i:{"label": f'{i}m', "style": {"fontSize": 16}} 
               for i in range(0, 4000, 500)},
        className="dbc"
    ),
    html.Div(id="output-div")
])

@app.callback(
    Output("title", "children"),
    Output("output-div", "children"),
    Input("country-dropdown", "value"),
    Input("Elevation Slider", "value"),
)

def elevation_table(country, elevation):
    if not country and elevation:
        raise PreventUpdate
    
    title = f"Ski Resorts in {country} with peaks above {elevation}M"
    
    df = ski_resorts.query("HighestPoint > @elevation and Country == @country")
    
    table = dash_table.DataTable(             
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict("records"),
        filter_action="native",
        sort_action="native",
        export_format="xlsx",
        style_header={
                'backgroundColor': 'rgb(30, 30, 30)',
                'color': 'lightgrey',
                'font-family': "Arial"
        },
        style_data={
                'backgroundColor': 'rgb(50, 50, 50)',
                'color': 'grey',
                'font-family': "Arial"
        }
    )

    return title, table 
    
if __name__ == "__main__":
    app.run_server(mode="inline", debug=True, port=8334)

# Assignment 2: Advanced Callbacks

Update your application from before to return either table or bar chart output based on user selection.

The app should not fire until the user manually triggers the callback.

* Your bar chart should have x="Resort", and y="DayPassPriceAdult", plotting the dataframe filtered by the country dropdown and price slider. 


In [6]:
from dash import dcc, html, dash_table, ctx
from jupyter_dash import JupyterDash
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template

import dash_bootstrap_components as dbc

import plotly.express as px
import pandas as pd

ski_resorts = pd.read_csv("../data/European_Ski_Resorts.csv").drop("Unnamed: 0", axis=1)

dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"

table_style = {
    'backgroundColor': 'rgb(50, 50, 50)',
    'color': 'grey',
    'font-family': "Arial"
}

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SLATE, dbc_css])

load_figure_template("SLATE")

app.layout = html.Div([
    html.H2(id="title", style={"text-align": "center"}),
    html.P("Select Options Below:"),
    dcc.Dropdown(
        id="country-dropdown", 
        options=ski_resorts["Country"].unique(),
        value="Andorra",
        className="dbc"
    ),
    dcc.Slider(
        id="Elevation Slider",
        min=0,
        max=4000,
        step=500,
        value=500,
        marks={
            i:{"label": f'{i}m', "style": {"fontSize": 16}} 
               for i in range(0, 4000, 500)},
        className="dbc"
    ),
    dcc.RadioItems(id="output-radio", options=["Chart", "Table"]),
    html.Button("Process Data", id="run-button"),
    html.Div(id="output-div")
])

@app.callback(
    Output("title", "children"),
    Output("output-div", "children"),
    Output("run-button", "n_clicks"),
    Input("country-dropdown", "value"),
    Input("Elevation Slider", "value"),
    Input("output-radio", "value"),
    Input("run-button", "n_clicks"),
)

def elevation_output(country, elevation, output_option, n_clicks):
    if not n_clicks:
        raise PreventUpdate

    title = f"Ski Resorts in {country} with peaks above {elevation}M"
    
    df = ski_resorts.query("HighestPoint > @elevation and Country == @country")
    
    if output_option == "Table":
    
        output = dash_table.DataTable(             
            columns=[{"name": i, "id": i} for i in df.columns],
            data=df.to_dict("records"),
            filter_action="native",
            sort_action="native",
            export_format="xlsx",
            style_header=table_style,
            style_data=table_style
        )
        
    else:
        output = dcc.Graph(figure=px.bar(df, x="Resort", y="DayPassPriceAdult"))
    
    n_clicks=0

    return title, output, n_clicks 
    
if __name__ == "__main__":
    app.run_server(
#         mode="inline", 
        debug=True, 
        port=8334,
    )

Dash app running on http://127.0.0.1:8334/
